In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Introduction

In this notebook, we are going to introduce you to the wonderful world of probabilistic inference using PyMC3!

In the spirit of sticking with simple complex examples, we are going to continue exploring the classic coin flip model.

## Probabilistic Model in PyMC3

Let's take that coin flip model with Beta-distributed $p$, and implement it in PyMC3 code.

In [ ]:
import pymc3 as pm

data = [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,]

with pm.Model() as model:
    p = pm.Beta("p", alpha=2, beta=2)
    data = pm.Bernoulli("data", p=p)

Just like that, we have specified a probabilistic model for coin flips!
Notice how, first of all, the syntax matches very closely
to how the probabilistic model is written in traditional math syntax:

$$ p \sim Beta(\alpha=2, \beta=2)$$
$$ data \sim Bernoulli(p=p)$$

and to job your memory, how we wrote it in the previous notebook using `scipy.stats`:

In [ ]:
from bayes_tutorial.solutions.simulation import coin_flip_generator_v2

coin_flip_generator_v2??

It should be clear that by expressing our model using the language of probability distributions,
we gain the ability to concisely write down statistical models.

Now, how does data come into play here? 